In [1]:
import pandas as pd, numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer   
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack
from scipy import sparse
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.feature_selection import SelectFromModel
from wordbatch.extractors import WordHash, WordBag
from wordbatch.models import FM_FTRL,FTRL

/usr/local/lib/python3.5/dist-packages/randomstate-1.14.0-py3.5-linux-x86_64.egg/randomstate/__init__.py:66: RandomStateDeprecationWarning: 
**End-of-life notification**

This library was designed to bring alternative generators to the NumPy 
infrastructure. It as been successful in advancing the conversation 
for a future implementation of a new random number API in NumPy which 
will allow new algorithms and/or generators. The next step
in this process is to separate the basic (or core RNG) from the 
functions that transform random bits into useful random numbers.
This has been implemented in a successor project  **randomgen** 
available on GitHub

https://github.com/bashtage/randomgen

or PyPi

https://pypi.org/project/randomstate/.

randomgen has a slightly different API, so please see the randomgen documentation

https://bashtage.github.io/randomgen.

  warnings.warn(DEPRECATION_MESSAGE, RandomStateDeprecationWarning)


In [2]:
column = "word_seg"
train = pd.read_csv('../input/new_data/train_set.csv')
test = pd.read_csv('../input/new_data/test_set.csv')
test_id = test["id"].copy()
y_all = (pd.Series(train['class'])-1).astype(int)

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [3]:
# trn_term_doc = sparse.load_npz('../feature/text_tifi/trn_term_doc.npz')
# trn_term_word = sparse.load_npz('../feature/text_tifi/trn_term_word.npz')
# trn_term_word_three = sparse.load_npz('../feature/text_tifi/trn_term_word_three.npz')
# train_feaure =  hstack([trn_term_doc,trn_term_word,trn_term_word_three]).tocsr()
# del trn_term_doc,trn_term_word,trn_term_word_three
# import gc
# gc.collect()
# trn_term_word_four = sparse.load_npz('../feature/text_tifi/trn_term_word_four.npz')
# # trn_term_word_five = sparse.load_npz('../feature/text_tifi/trn_term_word_five.npz')
# trn_char = sparse.load_npz('../feature/text_tifi/trn_char.npz')
# docLen  =pd.DataFrame( np.array(train[column].map(lambda x : len(x.split(" ")))/39759).reshape(-1,1))
# doclen_word = pd.DataFrame( np.array(train['article'].map(lambda x : len(x.split(" ")))/train[column].map(lambda x : len(x.split(" ")))).reshape(-1,1))
# docLen_test  =pd.DataFrame( np.array(test[column].map(lambda x : len(x.split(" ")))/39759).reshape(-1,1))
# doclen_word_test = pd.DataFrame( np.array(test['article'].map(lambda x : len(x.split(" ")))/test[column].map(lambda x : len(x.split(" ")))).reshape(-1,1))

In [4]:
# import gc
# del train, test
# gc.collect()

In [5]:
from keras.utils import np_utils
one_y_train=np_utils.to_categorical(y_all,num_classes=20)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
# train_feaure =  hstack([train_feaure, trn_term_word_four,trn_char,doclen_word]).tocsr()
# del trn_term_word_four,trn_char,doclen_word
# gc.collect()

In [7]:
# test_term_doc = sparse.load_npz('../feature/text_tifi/test_term_doc.npz')
# test_term_word = sparse.load_npz('../feature/text_tifi/test_term_word.npz')
# test_term_word_three = sparse.load_npz('../feature/text_tifi/test_term_word_three.npz')
# test_term_word_four = sparse.load_npz('../feature/text_tifi/test_term_word_four.npz')
# # test_term_word_five = sparse.load_npz('../feature/text_tifi/test_term_word_five.npz')
# test_char = sparse.load_npz('../feature/text_tifi/test_char.npz')
# test_festure =  hstack([test_term_doc,test_term_word,test_term_word_three,test_term_word_four,test_char,doclen_word_test]).tocsr()
# del test_term_doc,test_term_word,test_term_word_three,test_term_word_four,test_char,doclen_word_test
# gc.collect()

In [8]:
import pickle
svd_dir = '../feature/svd_5.pkl'
tifi_dir = '../feature/tifi_min4.pkl'
train_svd, test_svd= pickle.load(open(svd_dir,'rb'))
trn_term_doc, test_term_doc= pickle.load(open(tifi_dir,'rb'))
train_feaure = hstack((trn_term_doc, train_svd)).tocsr()
test_festure = hstack((test_term_doc, test_svd)).tocsr()

In [9]:
# X_train, X_valid, Y_train, Y_valid = train_test_split(train_feaure, y_all, test_size=0.15, random_state=42)

In [11]:
name = ["class_prob_%s"%i for i in range(1,20)]
one_y_train=np_utils.to_categorical(y_all,num_classes=20)



In [12]:
def ridge_f1_score(y_hat, data):
    y_true = data
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return f1_score(y_true, y_hat, average='macro')

In [15]:
from tqdm import tqdm
from sklearn.linear_model import Ridge
from tqdm import tqdm
from sklearn.model_selection import KFold

def kf_train(fold_cnt=3,rnd=1):
    now_nfold=0
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((102277,19)),np.zeros((102277,19))
    for train_index, test_index in kf.split(train_feaure):
        now_nfold+=1
        print ("now is {} fold".format(now_nfold))
        curr_x1, hold_out_x1 = train_feaure[train_index], train_feaure[test_index]
        curr_y, hold_out_y = one_y_train[train_index], one_y_train[test_index]
        for i in tqdm(range(19)):
            model = FM_FTRL(alpha=0.01, beta=0.01, L1=0.00001, L2=0.1, D=curr_x1.shape[1], alpha_fm=0.01, L2_fm=0.0, init_fm=0.01,
                            D_fm=100, e_noise=0.0001, iters=17, inv_link="identity", threads=4)

            model.fit(curr_x1, curr_y[:,i])
            test_pred[:,i] += model.predict(test_festure)
            ypred = model.predict(hold_out_x1)
            print ("{}line f1 score is {}".format(i, ridge_f1_score(ypred, hold_out_y[:,i])))
            train_pred[test_index,i] = ypred
    test_pred = test_pred / fold_cnt
    return train_pred, test_pred

In [16]:
train_pred,test_pred = kf_train(fold_cnt=10,rnd=4)

now is 1 fold


  0%|          | 0/19 [00:00<?, ?it/s]

Total e: 8110.007504816352
Total e: 6555.108862789603
Total e: 5840.125333879967
Total e: 5344.704239171087
Total e: 4959.871717240193
Total e: 4642.466116564272
Total e: 4371.208183286963
Total e: 4134.06072900261
Total e: 3922.9914169765398
Total e: 3732.953974727354
Total e: 3560.268957057683
Total e: 3402.311259578946
Total e: 3256.908827517338
Total e: 3122.415765938058
Total e: 2997.520673692213
Total e: 2881.2184311597284
Total e: 2772.550611909928


  5%|▌         | 1/19 [04:34<1:22:29, 275.00s/it]

0line f1 score is 0.7300770938175059
Total e: 4378.744427452854
Total e: 3296.0222940560384
Total e: 2820.728916890075
Total e: 2496.018127474563
Total e: 2248.5027678692854
Total e: 2048.3813857486994
Total e: 1880.8852714780978
Total e: 1737.1901983964956
Total e: 1611.6867482225996
Total e: 1500.8767822296777
Total e: 1402.1522161267378
Total e: 1313.5328668754926
Total e: 1233.5232822819075
Total e: 1160.9548408206879
Total e: 1094.8406297597191
Total e: 1034.4063224581835
Total e: 978.9765390969495


...

16line f1 score is 0.8592927272736401
Total e: 7825.9385397585775
Total e: 5442.55708016185
Total e: 4568.287218538973
Total e: 4007.263149039557
Total e: 3592.31565054961
Total e: 3263.80289933163
Total e: 2992.958658915123
Total e: 2763.2301299929036
Total e: 2564.411843984262
Total e: 2389.7169324011015
Total e: 2234.401139872254
Total e: 2095.276781769043
Total e: 1969.7193086567297
Total e: 1855.785662252722
Total e: 1751.9241536293023
Total e: 1656.8346290618383
Total e: 1569.4635353841488


 95%|█████████▍| 18/19 [1:20:46<04:29, 269.23s/it]

17line f1 score is 0.9221044860360832
Total e: 8549.751330677655
Total e: 6916.629326720462
Total e: 6120.104782476442
Total e: 5559.616613634102
Total e: 5123.314327212374
Total e: 4765.016012625153
Total e: 4460.281523022411
Total e: 4195.063992741188
Total e: 3960.3774855092042
Total e: 3750.499391755333
Total e: 3560.9103287655958
Total e: 3388.330383048669
Total e: 3230.204922877293
Total e: 3084.4335867762297
Total e: 2949.6325418218885
Total e: 2824.5327974403326
Total e: 2708.1120470707697


100%|██████████| 19/19 [1:25:10<00:00, 268.99s/it]

18line f1 score is 0.7362554757934826


In [19]:
test_pred[0]

array([-4.48980604e-02,  7.97121887e-03,  3.87572750e-04,  4.90017502e-02,
        8.87402939e-01,  3.48357102e-02,  5.40027465e-02,  1.75255933e-02,
        3.50844372e-02, -1.15918754e-01,  7.11722374e-02,  1.87814438e-02,
       -3.76947944e-02,  3.48229131e-02,  6.06087253e-03, -3.59507870e-03,
        8.93293604e-02, -3.91087109e-02,  8.48882779e-02])

In [20]:
import pickle
with open('../feature/stacking_pkl_file/10_fold_fm.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)